In [2]:
import glob
import pandas as pd
from pathlib import Path
import numpy as np
from skimage import io, measure, data

import napari
from napari_clusters_plotter import PlotterWidget
import napari_segment_blobs_and_things_with_membranes as nsbatwm

In [3]:
viewer = napari.Viewer()

napari.manifest -> 'napari-vedo-bridge' could not be imported: The name field in the manifest ('napari-clusters-plotter') must match the package name ('napari-vedo-bridge')


Assistant skips harvesting pyclesperanto as it's not installed.


In [4]:
root = r'./data'
image_files = glob.glob(r'E:\BiAPoL\Projects\napari-clusters-plotter\notebooks\data\**\**\*.tif')

In [5]:
features = [
    'label',
    'area_filled',
    'perimeter',
    'axis_major_length',
    'axis_minor_length',
    'eccentricity',
    'area',
    'intensity_mean',
    'intensity_max',
    'intensity_min',
    'intensity_std',
]

In [6]:
# calculate positions of images on grid
n_cols = np.sqrt(len(image_files))
n_rows = np.ceil(len(image_files) / n_cols)

spacing = 500
image_layers = []
labels_layers = []

for i, image_file in enumerate(image_files):
    row = i // n_cols
    col = i % n_cols
    image = io.imread(image_file)

    if len(image.shape) > 2:
        image = image[:, :, 0]

    # segmentation
    labels = np.asarray(nsbatwm.voronoi_otsu_labeling(image, spot_sigma=5))

    # measurements
    features = pd.DataFrame(measure.regionprops_table(labels, intensity_image=image, properties=features))

    image_layer = napari.layers.Image(data=image, name=Path(image_file).stem, translate=[spacing * col, spacing * row])
    labels_layer = napari.layers.Labels(data=labels, name=Path(image_file).stem + '_labels',
                                        translate=[spacing * col, spacing * row], features=features)
    
    image_layers.append(image_layer)
    labels_layers.append(labels_layer)

In [7]:
viewer.layers.clear()
for layer in image_layers:
    viewer.add_layer(layer)

for layer in labels_layers:
    viewer.add_layer(layer)

In [25]:
plotter_widget = PlotterWidget(viewer)
viewer.window.add_dock_widget(plotter_widget, area='right')

c:\Users\johan\mambaforge\envs\clusters-plotter\lib\site-packages\napari_matplotlib\base.py:46: FutureWarning: The `as_dict` kwarg has been deprecated since Napari 0.5.0 and will be removed in future version. You can use `get_theme(...).to_rgb_dict()`
  get_theme(napari_viewer.theme, as_dict=False)
c:\Users\johan\mambaforge\envs\clusters-plotter\lib\site-packages\napari_matplotlib\base.py:101: FutureWarning: The `as_dict` kwarg has been deprecated since Napari 0.5.0 and will be removed in future version. You can use `get_theme(...).to_rgb_dict()`
  theme = napari.utils.theme.get_theme(self.viewer.theme, as_dict=False)


In [1]:
from skimage import data, measure
import napari_segment_blobs_and_things_with_membranes as nsbatwm
import pandas as pd

image = data.human_mitosis()
labels = nsbatwm.voronoi_otsu_labeling(image, spot_sigma=5)
measurements = pd.DataFrame(measure.regionprops_table(labels, intensity_image=image, properties=['label', 'area']))